In [1]:
from LIMxCMBL.init import *
from LIMxCMBL.kernels import *

from scipy.interpolate import interp1d, LinearNDInterpolator
from scipy.integrate import quad, quad_vec, trapezoid

import jax
import jax.numpy as jnp

from interpax import interp2d, interp1d
from quadax import quadcc, quadgk

import sys
from os.path import isfile

from jax import config
config.update("jax_enable_x64", True)




Lambda_idx = 24#int(sys.argv[1])
n_external = 1201#int(sys.argv[2])
ell_idx = 58#int(sys.argv[3])


Lambda = Lambdas[Lambda_idx]

zmin = 1#float(sys.argv[4])
zmax = 5#float(sys.argv[5])

kernels = {}
kernels['CII'] = np.array(KI)
kernels['CO'] = np.array(KI_CO)
kernels['Lya'] = np.array(KI_Lya)
kernels['HI'] = np.array(KI_HI)


line_str = 'CII'# sys.argv[6]
print(line_str)
_KI = kernels[line_str]

oup_fname = '/scratch/users/delon/LIMxCMBL/I_auto/comb_'
oup_fname += '%s_zmin_%.1f_zmax_%.1f_Lambda_idx_%d_n_ext_%d_l_%d_jax_quad.npy'%(line_str,
                                                                                zmin, zmax, 
                                                                                Lambda_idx, 
                                                                                n_external,
                                                                                ell_idx)


print(oup_fname)

chimin = ccl.comoving_angular_distance(cosmo, 1/(1+zmin))
chimax = ccl.comoving_angular_distance(cosmo, 1/(1+zmax))


external_chis = np.linspace(chimin*(1+1e-8), chimax*(1 - 1e-8), n_external)
dchi = np.mean(np.diff(external_chis))
print('external chi spacing', dchi)

inner_dkparp_integral = np.load('/oak/stanford/orgs/kipac/users/delon/LIMxCMBL/inner_dkparp_integral.npy')
 

CII
/scratch/users/delon/LIMxCMBL/I_auto/comb_CII_zmin_1.0_zmax_5.0_Lambda_idx_24_n_ext_1201_l_58_jax_quad.npy
external chi spacing 3.7943019412119474


In [2]:
@jax.jit
def f_KILo(chi, external_chi, Lambda):
    return (Lambda / jnp.pi 
            * jnp.interp(x = chi, xp = chis, 
                         fp = _KI, left = 0, right = 0) 
            * jnp.sinc(Lambda * (external_chi - chi) / np.pi))

inner_dkparp_integral = jnp.array(inner_dkparp_integral.astype(np.float64))

2025-03-15 18:40:49.054327: W external/xla/xla/service/gpu/nvptx_compiler.cc:765] The NVIDIA driver's CUDA version is 12.4 which is older than the ptxas CUDA version (12.6.85). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [3]:
external_chis[58]

3625.443077299837

In [4]:
chimin*(1+1e-8) + 58 * (chimax*(1-1e-8) - chimin*(1+1e-8)) / (n_external - 1)

3625.443077299837

In [5]:
chimax

7958.535973749241

In [6]:
chimin + (dchi * (n_external-1))

7958.535860110146

In [7]:
@jax.jit
def f_aux(_chib, chi, chip):
    _delta = jnp.abs(1 - chi/_chib)
    _delta = jnp.where(_delta < 1e-6, 1e-6, 
                     jnp.where(_delta > 0.7, 0.7, _delta))

    cross_integrand_1 = (2 * jnp.interp(x = chi, xp = chis, fp = _KI, left = 0, right = 0) 
                       * interp2d(xq = _chib, yq=jnp.log(_delta), 
                           x = chibs, y = jnp.log(deltas), f=inner_dkparp_integral[ell_idx],
                           method='linear',) 
                       / (_chib**2))

    cross_integrand_1 *= f_KILo(2*_chib - chi, 
                                        external_chi = chip,
                                        Lambda=Lambda)
    return cross_integrand_1
@jax.jit
def f_integrand(_chib, chi_idx, chip_idx, ell_idx):
    #by construction chimin < exteranl_chis < chimax 
    #I Lo + Lo I
    chi = chimin*(1+1e-8) + chi_idx * (chimax*(1-1e-8) - chimin*(1+1e-8)) / (n_external - 1)
    chip = chimin*(1+1e-8) + chip_idx * (chimax*(1-1e-8) - chimin*(1+1e-8)) / (n_external - 1)

    cross_integrand_1 = jnp.where((chimin <= 2*_chib - chi) & (2*_chib - chi <= chimax),
                                  f_aux(_chib, chi, chip),
                                  0)
            
    cross_integrand_2 = jnp.where((chimin <= 2*_chib - chip) & (2*_chib - chip <= chimax),
                                  f_aux(_chib, chip, chi),
                                  0)
            
    cross_integrand = cross_integrand_1 + cross_integrand_2

    #LoLo
    plus = _chib*(1+deltas)
    mins = _chib*(1-deltas)
    _idxs = (chimin < plus) & (plus < chimax) & (chimin < mins) & (mins < chimax)
    
    LoLo_integrand  = jnp.where(_idxs,
                               f_KILo(plus, 
                                      external_chi = chi,
                                      Lambda=Lambda) 
                                * f_KILo(mins, 
                                         external_chi = chip,
                                         Lambda=Lambda),
                               0)
    LoLo_integrand += jnp.where(_idxs,
                               f_KILo(mins, 
                                      external_chi = chi,
                                      Lambda=Lambda) 
                                * f_KILo(plus, 
                                         external_chi = chip,
                                         Lambda=Lambda),0)
    LoLo_integrand *= (2 / _chib) #(d)
    LoLo_integrand = jnp.einsum('d,d->d', LoLo_integrand, deltas)
    LoLo_integrand = jnp.einsum('d,d->d', LoLo_integrand, 
                                interp1d(xq = _chib,
                                         x = chibs, f=inner_dkparp_integral[ell_idx],
                                         method='linear',))
    
    LoLo_integrand = jnp.trapezoid(x = np.log(deltas), y = LoLo_integrand, axis=-1)
    return LoLo_integrand - cross_integrand

In [8]:
@jax.jit
def OLDf_integrand(_chib, ell_idx):
    #by construction chimin < exteranl_chis < chimax 
    #I Lo + Lo I
    
    _delta = jnp.abs(1 - external_chis/_chib) #(n_ext)
    _delta = jnp.where(_delta < 1e-6, 1e-6, 
                     jnp.where(_delta > 0.7, 0.7, _delta))
    
    _idx = ((chimin <= 2*_chib - external_chis) 
            & (2*_chib - external_chis <= chimax)) #(n_ext)
    
    cross_integrand = (2 * jnp.interp(x = external_chis, xp = chis, fp = _KI, left = 0, right = 0) 
                       * interp2d(xq = _chib, yq=jnp.log(_delta), 
                           x = chibs, y = jnp.log(deltas), f=inner_dkparp_integral[ell_idx],
                           method='linear',) 
                       / (_chib**2))
    
    cross_integrand = jnp.where(_idx.reshape(-1, 1),
                               cross_integrand.reshape(-1, 1)
                               * f_KILo(2*_chib - external_chis.reshape(-1,1), 
                                        external_chi = external_chis.reshape(1, -1), 
                                        Lambda=Lambda),
                                0)
    cross_integrand = cross_integrand + jnp.moveaxis(cross_integrand, -1, -2) # the two cross terms are just from switching chi and chi'
    #LoLo
    plus = _chib*(1+deltas)
    mins = _chib*(1-deltas)
    _idxs = (chimin < plus) & (plus < chimax) & (chimin < mins) & (mins < chimax)
    
    LoLo_integrand  = jnp.where(_idxs,
                               f_KILo(plus, 
                                      external_chi = external_chis.reshape(-1, 1, 1), 
                                      Lambda=Lambda) 
                                * f_KILo(mins, 
                                         external_chi = external_chis.reshape(1, -1, 1), 
                                         Lambda=Lambda),
                               0)
    LoLo_integrand += jnp.where(_idxs,
                               f_KILo(mins, 
                                      external_chi = external_chis.reshape(-1, 1, 1), 
                                      Lambda=Lambda) 
                                * f_KILo(plus, 
                                         external_chi = external_chis.reshape(1, -1, 1), 
                                         Lambda=Lambda),0)
    LoLo_integrand *= (2 / _chib) #(x,y,d)
    LoLo_integrand = jnp.einsum('xyd,d->xyd', LoLo_integrand, deltas)
    LoLo_integrand = jnp.einsum('xyd,d->xyd', LoLo_integrand, 
                                interp1d(xq = _chib,
                                         x = chibs, f=inner_dkparp_integral[ell_idx],
                                         method='linear',))
    
    LoLo_integrand = jnp.trapezoid(x = np.log(deltas), y = LoLo_integrand, axis=-1)
    return LoLo_integrand - cross_integrand



In [9]:
f_integrand((chimin + chimax)/2, 58, 59, ell_idx = ell_idx)

Array(2.84693898e-11, dtype=float64)

In [10]:
OLDf_integrand((chimin + chimax)/2, ell_idx = ell_idx)[58, 59]

Array(2.84693898e-11, dtype=float64)

In [11]:
from tqdm import trange

In [12]:
oup = np.zeros((n_external, n_external))

for chi_idx in range(n_external):
    for chip_idx in trange(n_external):
        res, tmp = quadcc(f_integrand, jnp.hstack([10, 
                                           jnp.linspace(chimin, chimax, 100), 
                                           chimax_sample]),
                  epsabs = 0.0, epsrel = 1.4e-8, 
                  order = 256, max_ninter=650, args=(58, 59, ell_idx,),
                  full_output = True)
        oup[chi_idx,chip_idx] = oup[chip_idx, chi_idx] = res
        assert(tmp[3]['ninter']< 650)
        print('intervals used', tmp[3]['ninter'])



  0%|          | 1/1201 [00:14<4:51:22, 14.57s/it]

intervals used 136


  0%|          | 2/1201 [00:19<3:02:04,  9.11s/it]

intervals used 136


  0%|          | 3/1201 [00:24<2:25:38,  7.29s/it]

...

intervals used 136


  2%|▏         | 20/1201 [01:55<1:44:47,  5.32s/it]

intervals used 136


  2%|▏         | 20/1201 [01:57<1:56:04,  5.90s/it]


KeyboardInterrupt: 

this is going to take way too damn long